**Things to extract**

~~- Utility Name~~
~~- Issued Date~~

~~- Company Name~~
~~- Street~~
~~- City~~
~~- State~~
~~- Zip~~


~~- Electric Choice ID#~~
~~- Rate Code~~

~~- Usage~~
~~- Supplier~~


In [123]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 243 kB 1.6 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [124]:
import pdfplumber
import pprint
import pandas as pd
import openpyxl

In [71]:
### FIRST PAGE ###

def setAddressBoundingBox(page):
    
    #get page height and width for box coordinate calculations
    page_height = page.height
    page_width = page.width
    
    #address box is near second top quarter
    top_half = page.height/2
    top_quarter = page.height/4
    second_quarter = top_half - top_quarter
    
    #coordinates of the address box
    address_box_left = (page.width//2) - 20
    address_box_top = second_quarter - 75
    address_box_right = page.width
    address_box_bottom = (page.height//2) -215
    
    address_bounding_box = (address_box_left,address_box_top,address_box_right,address_box_bottom)
    
    return address_bounding_box

def extractAddressBoxText(page, address_bounding_box):
    
    address_extract_text = page.crop(address_bounding_box).extract_text(x_tolerance=1)
    
    return address_extract_text

def setCompanyName(address_extract):
    
    company_name_text = address_extract.splitlines()[0]
    
    return company_name_text
    
def setStreet(address_extract):

    street_text = address_extract.splitlines()[1]
    
    return street_text

def setCityStateZIP(address_extract):
    
    city_state_zip_text = address_extract.splitlines()[2]
    city_state_zip_text = city_state_zip_text.replace(',', '')
    
    city_text = city_state_zip_text.split(' ')[0]
    state_text =  city_state_zip_text.split(' ')[1]
    zip_code_text =  city_state_zip_text.split(' ')[2]
    
    return city_text, state_text, zip_code_text

def setAccNum(address_extract): 
    
    acc_num_text = address_extract.splitlines()[3]
    acc_num_text = acc_num_text.split('#')[1]
    acc_num_text = acc_num_text.strip()
    
    return acc_num_text

def setIssuedDate(address_extract):
    
    issued_date_text = address_extract.splitlines()[4]
    issued_date_text = issued_date_text.split(':')[1]
    issued_date_text = issued_date_text.strip()
    
    return issued_date_text

def setElectricSupplyBoundingBox(page):
    
    top_half = page.height/2
    top_quarter = page.height/4
    second_quarter = top_half - top_quarter

    electric_supply_box_top  = second_quarter - 75
    electric_supply_box_bottom = (page.height//2) - 215
    electric_supply_box_left = (first_page.width//2) - 130
    electric_supply_box_right = (first_page.width//2)
    
    electric_supply_bounding_box = (electric_supply_box_left,electric_supply_box_top,electric_supply_box_right,electric_supply_box_bottom)
    
    return electric_supply_bounding_box

def extractElectricSupplyBoxText(page, electric_supply_bounding_box):
    
    electric_supply_extract_text = page.crop(electric_supply_bounding_box).extract_text(x_tolerance=1)
    
    return electric_supply_extract_text

def setElectricSupplier(electric_supply_extract_text):
    
    electric_supplier_text = electric_supply_extract.splitlines()[1]
    
    return electric_supplier_text

def setElectricChoiceID(electric_supply_extract_text):
    
    electric_choice_id_text = electric_supply_extract.splitlines()[4]
    electric_choice_id_text = electric_choice_id_text.split('Electric Choice ID:')[1].strip()
    
    return electric_choice_id_text

def setUtilityName(page):
    
    page_extract = page.extract_text(x_tolerance=1, y_tolerance=1)
    utility_name_text = page_extract.splitlines()[-6]
    
    return utility_name_text

### SECOND PAGE ###

def setRateBoundingBox(page):
    
    rate_box_left = 20
    rate_box_top = (page.width//4) + 5
    rate_box_right = (page.width//2) - 100
    rate_box_bottom = (page.width//4) + 50
    rate_bounding_box = (rate_box_left, rate_box_top, rate_box_right, rate_box_bottom)
    
    return rate_bounding_box
    
def extractRateBoxText(page, rate_bounding_box):
    
    rate_extract_text = second_page.crop(rate_bounding_box).extract_text(x_tolerance = 1)
    
    return rate_extract_text

def setRate(rate_extract_text):

    #gets the first line after the word 'Service' and remove the leading whitespace and '-'
    rate_text = rate_extract_text.split('Service')[1].splitlines()[0].replace('-','',1).strip()
    
    if 'TOU -' in rate_text:
        
        #remove TOU and leading '-'
        rate_text = rate_text.split('TOU')[1].replace('-','',1).strip()
    
    return rate_text

def setUsageBoundingBox(page):
    
    usage_box_left = (page.width//2) - 130
    usage_box_top = (page.height//2) - 180
    usage_box_right = (page.width//2) - 20
    usage_box_bottom = (page.height//2) - 150
    usage_bounding_box = (usage_box_left,usage_box_top,usage_box_right,usage_box_bottom)
    
    return usage_bounding_box

def extractUsageBoxText(page, usage_bounding_box):
    
    usage_extract_text = page.crop(usage_bounding_box).extract_text()
    
    return usage_extract_text

def setUsage(usage_extract_text):
    
    usage_text = usage_extract_text.splitlines()[0]
    #usage_text = usage_extract_text.replace('=', '')
    #usage_text = usage_extract_text.split('used')[0]
    
    return usage_text



    

    

In [79]:
def createInitialDFDict(bill_ls):
    
    bills_dict = {}
    
    for item in bill_ls:
        
        bills_dict[item] = eval(item)
    
    print(bills_dict)
    

In [132]:
pp = pprint.PrettyPrinter(indent = 4)

file_name = input("Name of PDF file: ")

# file_name = 'PII Example 1'
#with pdfplumber.open('Bills/'+ file_name + '.pdf') as pdf:

company_name = ''
street = ''
city = ''
state = ''
zip_code = ''
issued_date = ''
electric_supplier = ''
electric_choice_id = ''
rate = ''
usage = ''

bill_dict = {}

with pdfplumber.open(f'Bills/{file_name}.pdf') as pdf:
    first_page = pdf.pages[0]
    second_page = pdf.pages[1]
    
    #extracting from address box
    address_extract = extractAddressBoxText(first_page, setAddressBoundingBox(first_page))
    
    company_name = setCompanyName(address_extract)
    street = setStreet(address_extract)
    city, state, zip_code = setCityStateZIP(address_extract)
#     acc_num = setAccNum(address_extract)
    issued_date = setIssuedDate(address_extract)
   
    #extracting from electric supply box
    electric_supply_extract = extractElectricSupplyBoxText(first_page, setElectricSupplyBoundingBox(first_page))
    
    electric_supplier = setElectricSupplier(electric_supply_extract)
    electric_choice_id = setElectricChoiceID(electric_supply_extract)
    
    #extracting utility name from the bottom of the page
    utility_name = setUtilityName(first_page)
    
    #extracting from rate box
    rate_extract = extractRateBoxText(second_page, setRateBoundingBox(second_page))
    rate = setRate(rate_extract)

    #extracting from usage
    usage_extract = extractUsageBoxText(second_page, setUsageBoundingBox(second_page))
    usage = setUsage(usage_extract)
    
# bill_var_ls = [company_name, street, city, state, zip_code, issued_date, electric_supplier, electric_choice_id, rate, usage]

# createInitialDFDict(bill_var_ls)

bill_dict = dict(utility = utility_name, issued_date = issued_date, company = company_name, 
                 street = street, city = city, state = state, zip_code = zip_code, 
                 electric_choice_id = electric_choice_id, 
                 rate_code = rate, electric_supplier = electric_supplier, usage = usage)

# pp.pprint(bill_dict, sort_dicts = False)
#10.22.20 E, 11.18.20 E format looks slightly different so doesn't work

bill_dict


Name of PDF file: 11.23.20 - E


{'utility': 'BGE',
 'issued_date': 'November 23, 2020',
 'company': 'Pharmaceutics International Inc',
 'street': '10947 Golden-West Dr *PS',
 'city': 'Cockeysville',
 'state': 'MD',
 'zip_code': '21030',
 'electric_choice_id': '5700075826',
 'rate_code': 'Schedule GL',
 'electric_supplier': 'Constellation New Energy, Inc',
 'usage': '89744'}

In [133]:
bill_df = pd.DataFrame([bill_dict])
bill_df

,utility,issued_date,company,street,city,state,zip_code,electric_choice_id,rate_code,electric_supplier,usage
0,BGE,"November 23, 2020",Pharmaceutics International Inc,10947 Golden-West Dr *PS,Cockeysville,MD,21030,5700075826,Schedule GL,"Constellation New Energy, Inc",89744


In [134]:
bill_df.to_excel("output.xlsx")